Sử dụng LightAutoML giải quyết bài toán binary classification của Titanic.

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score

from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

In [ ]:
# Tải bộ dữ liệu Titanic từ Seaborn (rất tiện lợi)
data = sns.load_dataset('titanic')

# Chúng ta chỉ lấy các cột cơ bản, LAMA sẽ tự xử lý phần còn lại
# LAMA sẽ tự động xử lý NaNs trong 'age' và 'embarked'
data = data[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]

# Chia dữ liệu: 80% train, 20% test
# 'stratify' để đảm bảo tỷ lệ Sống/Chết trong 2 tập là như nhau
train_df, test_df = train_test_split(data, 
                                     test_size=0.2, 
                                     stratify=data['survived'], 
                                     random_state=42)

print(f"Kích thước tập Train: {train_df.shape}")
print(f"Kích thước tập Test: {test_df.shape}")

In [ ]:
# --- 2. Thiết lập LightAutoML ---

# Bước 2a: Định nghĩa Task
# 'binary' = phân loại nhị phân
# 'metric' = 'roc_auc' là thước đo tiêu chuẩn cho bài toán này
task = Task('binary', metric='roc_auc')

# Bước 2b: Định nghĩa Roles
# Chỉ cần nói cho LAMA biết đâu là cột mục tiêu (target)
# LAMA sẽ tự động suy ra các cột còn lại là features.
roles = {'target': 'survived'}

In [ ]:
# --- 3. Tạo và Huấn luyện (Train) AutoML ---

# Đặt thời gian timeout (tính bằng giây)
# 300 giây = 5 phút. LAMA sẽ cố gắng làm tốt nhất trong 5 phút.
# Bạn có thể tăng lên nếu muốn kết quả tốt hơn.
TIMEOUT = 300 

# Khởi tạo mô hình AutoML
# 'TabularAutoML' là preset tốt nhất cho dữ liệu dạng bảng
automl = TabularAutoML(task=task, 
                       timeout=TIMEOUT,
                       cpu_limit=4) # Giới hạn 4 CPU cores

# Huấn luyện mô hình!
# .fit_predict sẽ huấn luyện trên train_df
# và trả về OOF (Out-of-Fold) predictions.
print("Bắt đầu huấn luyện AutoML...")
oof_predictions = automl.fit_predict(train_df, roles=roles)

In [ ]:
# --- 4. Đánh giá (Evaluate) trên tập Test ---

# Sử dụng mô hình đã huấn luyện để dự đoán trên tập test_df
print("Bắt đầu dự đoán trên tập Test...")
test_predictions = automl.predict(test_df)

# test_predictions.data là một array (N, 2)
# Cột 0 là xác suất cho class 0 (Chết)
# Cột 1 là xác suất cho class 1 (Sống)
test_probs_class_1 = test_predictions.data[:, 1]

# Lấy ra nhãn thật
y_true = test_df['survived']

# Tính toán Accuracy
# Chuyển đổi xác suất > 0.5 thành 1, ngược lại là 0
y_pred = (test_probs_class_1 > 0.5).astype(int)

# In kết quả
auc_score = roc_auc_score(y_true, test_probs_class_1)
acc_score = accuracy_score(y_true, y_pred)

print("--- KẾT QUẢ ĐÁNH GIÁ ---")
print(f"ROC-AUC trên tập Test: {auc_score:.4f}")
print(f"Accuracy trên tập Test: {acc_score:.4f}")

In [ ]:
# Lấy bảng điểm quan trọng của feature
feature_scores = automl.get_feature_scores()

# In ra 10 feature quan trọng nhất
print("\n--- Feature Importance ---")
print(feature_scores.head(10))